In [ ]:
import requests
import json
import time

url_base = "https://api.card-gorilla.com:8080/v1/cards"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    "Referer": "https://www.card-gorilla.com/search/condition?cate=CRD",
    "Accept": "application/json",
    "Origin": "https://www.card-gorilla.com",
}

all_idx = []
page = 1
per_page = 100  # 100으로 시작 (실패하면 50으로 낮춰서 다시 돌리기)

while True:
    params = {
        "p": page,
        "perPage": per_page,
        "cate": "CRD",
        "is_discon": 0,
    }

    try:
        res = requests.get(url_base, params=params, headers=headers, timeout=10)

        print(f"페이지 {page} 상태코드: {res.status_code}")

        if res.status_code != 200:
            print("실패 → perPage 낮춰서 재시도 하거나 헤더/Cookie 확인")
            break

        data = res.json()
        cards = data.get("data", [])
        total = data.get("total", 0)

        if not cards:
            break

        page_idx = [c["idx"] for c in cards if "idx" in c]
        all_idx.extend(page_idx)

        print(f"페이지 {page}: {len(page_idx)}개 | 누적 {len(all_idx)} / {total}")

        if len(cards) < per_page:
            break

        page += 1
        time.sleep(1.2)

    except Exception as e:
        print(f"페이지 {page} 에러: {e}")
        break

# 결과 저장
with open("all_card_idx.json", "w", encoding="utf-8") as f:
    json.dump(all_idx, f, ensure_ascii=False)

print(f"\n완료! 총 {len(all_idx)}개 idx 모음")
print("파일: all_card_idx.json")
if all_idx:
    print("처음 10개:", all_idx[:10])
    print("마지막 10개:", all_idx[-10:])

In [ ]:
import requests
import json
import time

# API 베이스 URL
BASE_URL = "https://api.card-gorilla.com:8080/v1/cards"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    "Referer": "https://www.card-gorilla.com/",
    "Accept": "application/json",
    "Origin": "https://www.card-gorilla.com",
}

# 모은 idx 리스트 불러오기
with open("all_card_idx.json", "r", encoding="utf-8") as f:
    idx_list = json.load(f)

print(f"총 {len(idx_list)}개 idx 로드됨")

full_data = []

for i, idx in enumerate(idx_list, 1):
    url = f"{BASE_URL}/{idx}"
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            print(f"[{i}/{len(idx_list)}] idx {idx} 실패: {response.status_code}")
            continue
        
        card_data = response.json()
        
        # 핵심 혜택 추출
        extracted = {
            "idx": idx,
            "name": card_data.get("name", "이름없음"),
            "corp": card_data.get("corp", {}).get("name", "알수없음"),
            "annual_fee": card_data.get("annual_fee_basic", ""),
            "top_benefit": card_data.get("top_benefit", []),
            "key_benefit": card_data.get("key_benefit", []),
            "search_benefit": card_data.get("search_benefit", []),
            "event": card_data.get("event", {}),
            "pre_month_money": card_data.get("pre_month_money", 0)
        }
        
        full_data.append(extracted)
        
        print(f"[{i}/{len(idx_list)}] idx {idx} 완료: {extracted['name']} → top 혜택 {len(extracted['top_benefit'])}개, key 혜택 {len(extracted['key_benefit'])}개")
        
        if i % 50 == 0:
            with open("midway_full_data.json", "w", encoding="utf-8") as f:
                json.dump(full_data, f, ensure_ascii=False, indent=2)
            print("중간 저장 완료")
        
        time.sleep(1.5)  # 서버 부하 방지
        
    except Exception as e:
        print(f"idx {idx} 에러: {e}")
        time.sleep(5)

# 최종 저장
with open("card_gorilla_full_benefits_1075.json", "w", encoding="utf-8") as f:
    json.dump(full_data, f, ensure_ascii=False, indent=2)

print(f"\n완료! {len(full_data)}개 카드 상세 혜택 데이터 저장됨")